In [ ]:
from typing import Dict
import pandas as pd
import preprocessing as pre


def run_preprocess_pipeline(
    path: str,
    use_cols: list[str],
    missing_cols: list[str],
    rename_map: dict[str, str] | None = None
) -> Dict[str, pd.DataFrame]:
    """
    공용 전처리 파이프라인
    - 어떤 기상 데이터 엑셀에도 적용 가능
    - 결과는 시트별 DataFrame dict 반환
    """

    raw_sheets = pd.read_excel(path, sheet_name=None)
    processed = {}

    for region, df in raw_sheets.items():
        df = df.copy()

        # 컬럼 정리
        df.columns = df.columns.str.strip().str.replace(" ", "")

        # 컬럼 선택
        valid_cols = [c for c in use_cols if c in df.columns]
        df = df[valid_cols]

        # 결측치 처리
        for col in missing_cols:
            if col in df.columns:
                df = pre.handle_missing(df, col)

        # 컬럼 rename (선택)
        if rename_map:
            df = df.rename(columns=rename_map)

        # 지역 컬럼
        df["region"] = region

        processed[region] = df

    return processed

In [ ]:
#강수량 전처리 전용 코드

RAIN_USE_COLS = [
    "지점번호",
    "지점명",
    "일시",
    "강수량(mm)",
    "일최다강수량(mm)"
]

RAIN_RENAME_MAP = {
    "강수량(mm)": "rainfall",
    "일최다강수량(mm)": "max_rainfall"
}

In [ ]:
#일조일사 칼럼

SUN_USE_COLS = [
    '지점번호',
    '지점명',
    '일시',
    '일조합(hr)',
    '일조율(%)',
]


In [ ]:
#기온
TEMP_USE_COLS = [
    '지점번호',
    '지점명',
    '일시',
    '평균기온(℃)',
    '평균최고기온(℃)',
    '최고기온(℃)',
    '평균최저기온(℃)',
    '최저기온(℃)',
]

In [ ]:
#풍속
WIND_USE_COLS = [
    '지점번호',
    '지점명',
    '일시',
    '평균풍속(m/s)',
    '최대풍속(m/s)',
    '최대풍속풍향(deg)',
    '최대순간풍속(m/s)',
    '최대순간풍속풍향(deg)'
]

In [ ]:
#이건 만들었던 전처리 함수 코드.

import preprocessing as pre
#만들어 뒀던 결측치 처리 py파일 불러움.

processed_dfs = {}
#빈 딕셔너리 만드는 건 시트별로 전처리된 DataFrame을 저장하기 위해 사용.

use_cols = [
    '지점번호',
    '지점명',
    '일시',
    '강수량(mm)',
    '일최다강수량(mm)'
]

print("총 시트 수:", len(df1))

for sheet, df in df1.items():
    print("처리 중:", sheet) #엑셀 시트가 처리될 때마다 출력

    df.columns = df.columns.str.strip().str.replace(" ", "") #컬럼 명 정리
    #특정 컬럼만 추려내서 데이터화 -원본 dfs 살리기 위해 copy 씀.
    df_use = df[[c for c in use_cols if c in df.columns]].copy() #컬럼 
    processed_dfs[sheet] = df_use
    #전처리 된 df_use를 sheet라는 키로 저장해라.

    # 결측치 처리
    for col in ["강수량(mm)","일최다강수량(mm)"]:
            if col in df_use.columns:
                df_use = pre.handle_missing(df_use, col)
    processed_dfs[sheet] = df_use


총 시트 수: 14
처리 중: 강원도(강릉)
처리 중: 강원도(대관령)
처리 중: 강원도(동해)
처리 중: 강원도(북강릉)
처리 중: 강원도(북춘천)
처리 중: 강원도(속초)
처리 중: 강원도(영월)
처리 중: 강원도(원주)
처리 중: 강원도(인제)
처리 중: 강원도(정선군)
처리 중: 강원도(철원)
처리 중: 강원도(춘천)
처리 중: 강원도(태백)
처리 중: 강원도(홍천)


,지점번호,지점명,일시,강수량(mm),일최다강수량(mm)
0,216,태백,2020-01,79.4,30.0
1,216,태백,2020-02,36.1,23.6
2,216,태백,2020-03,24.8,15.6
3,216,태백,2020-04,46.4,23.3
4,216,태백,2020-05,102.4,31.7
...,...,...,...,...,...
67,216,태백,2025-08,43.4,15.4
68,216,태백,2025-09,175.1,67.7
69,216,태백,2025-10,277.4,53.2
70,216,태백,2025-11,9.8,4.0


In [ ]:
#전처리 과정 거친 데이터를 엑셀 시트로 저장
#추후 시각화 과정에서 정제된 데이터를 바로 쓰기 위함.

output_path = ".xlsx"

with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    for sheet, df in processed_dfs.items():
        df.to_excel(
            writer,
            sheet_name=sheet,
            index=False
        )